In [1]:
#| default_exp xml

# XML

> Concise generation of XML.

In [36]:
#| export
from fastcore.utils import *

from dataclasses import dataclass, asdict
import types
from functools import partial
from html import escape

In [3]:
from IPython.display import Markdown
from pprint import pprint

In [4]:
#| export
def _attrmap(o):
    o = dict(htmlClass='class', cls='class', klass='class', fr='for', htmlFor='for').get(o, o)
    return o.lstrip('_').replace('_', '-')

In [5]:
#| export
named = set('a button form frame iframe img input map meta object param select textarea'.split())

In [6]:
#|export
class XT(list): patch

In [7]:
#| export
def xt(tag:str, *c, **kw):
    "Create an XML tag structure `[tag,children,attrs]` for `toxml()`"
    if len(c)==1 and isinstance(c[0], types.GeneratorType): c = tuple(c[0])
    kw = {_attrmap(k):str(v) for k,v in kw.items()}
    if tag in named and 'id' in kw and 'name' not in kw: kw['name'] = kw['id']
    return XT([tag.lower(),c,kw])

In [8]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'A', 'Link', 'Nav',
'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details',
'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt, o.lower())

The main HTML tags are exported as `xt` partials.

Attributes are passed as keywords. Use 'klass' and 'fr' instead of 'class' and 'for', to avoid Python reserved word clashes.

In [9]:
samp = Html(
    Head(Title('Some page')),
    Body(Div(P('Some text'), Input(name='me'), Img(src="filename"), klass='myclass'))
)
pprint(samp)

['html',
 (['head', (['title', ('Some page',), {}],), {}],
  ['body',
   (['div',
     (['p', ('Some text',), {}],
      ['input', (), {'name': 'me'}],
      ['img', (), {'src': 'filename'}]),
     {'class': 'myclass'}],),
   {}]),
 {}]


For tags that have a `name` attribute, it will be set to the value of `id` if not provided explicitly:

In [10]:
pprint(Button(id='btn'))

['button', (), {'id': 'btn', 'name': 'btn'}]


In [11]:
#| export
def Checkbox(value:bool=False, **kw):
    checked = {"checked":"1"} if value else {}
    return Input(type="checkbox", **checked, **kw)

In [12]:
#| export
def Hidden(value:str="", **kw):
    return Input(type="hidden", value=value, **kw)

In [13]:
#| export
voids = set('area base br col command embed hr img input keygen link meta param source track wbr'.split())

In [14]:
#| export
def to_xml(elm, lvl=0):
    "Convert `xt` element tree into an XML string"
    if isinstance(elm, tuple): return '\n'.join(to_xml(o) for o in elm)
    if hasattr(elm, '__xt__'): elm = elm.__xt__()
    sp = ' ' * lvl
    if not isinstance(elm, list):
        if isinstance(elm, str): elm = escape(elm)
        return f'{elm}\n'

    tag,cs,attrs = elm
    stag = tag
    if attrs:
        sattrs = (f'{k}="{escape(str(v), quote=False)}"' for k,v in attrs.items())
        stag += ' ' + ' '.join(sattrs)
    
    cltag = '' if tag in voids else f'</{tag}>'
    if not cs: return f'{sp}<{stag}>{cltag}\n'
    res = f'{sp}<{stag}>\n'
    res += ''.join(to_xml(c, lvl=lvl+2) for c in cs)
    if tag not in voids: res += f'{sp}{cltag}\n'
    return res

In [15]:
print(to_xml(samp))

<html>
  <head>
    <title>
Some page
    </title>
  </head>
  <body>
    <div class="myclass">
      <p>
Some text
      </p>
      <input name="me">
      <img src="filename">
    </div>
  </body>
</html>



In [16]:
@patch
def _repr_markdown_(self:XT):
    try: from IPython import display
    except ImportError: return repr(self)
    return f'```html\n{to_xml(self)}\n```'

Automatic syntax highlighted output in notebooks:

In [17]:
samp

```html
<html>
  <head>
    <title>
Some page
    </title>
  </head>
  <body>
    <div class="myclass">
      <p>
Some text
      </p>
      <input name="me">
      <img src="filename">
    </div>
  </body>
</html>

```

In [25]:
#| export
def set_val(tag, attr, val):
    if attr.get('type', '') in ('checkbox','radio'):
        if val: attr['checked'] = '1'
        else: attr.pop('checked', '')
    else: attr['value'] = val

In [32]:
#| export
def fill_form(form, obj):
    "Modifies form in-place and returns it"
    inps = {attrs['id']:(tag,attrs) for tag,c,attrs in form[1] if 'id' in attrs}
    for nm,val in asdict(obj).items():
        if nm in inps:
            tag,attr = inps[nm]
            set_val(tag, attr, val)
    return form

In [33]:
@dataclass
class TodoItem:
    title:str; id:int; done:bool
                
todo = TodoItem(id=2, title="Profit", done=True)
form = Form(Input(id="title"), Checkbox(id="done"),
            Hidden(id="id"), Button("Save"))
fill_form(form, todo)

```html
<form>
  <input id="title" name="title" value="Profit">
  <input type="checkbox" id="done" name="done" checked="1">
  <input type="hidden" value="2" id="id" name="id">
  <button>
Save
  </button>
</form>

```

In [39]:
#|export
def fill_dataclass(src, dest):
    "Modifies dataclass in-place and returns it"
    for nm,val in asdict(src).items(): setattr(dest, nm, val)
    return dest

In [40]:
nt = TodoItem('', 0, False)
fill_dataclass(todo, nt)
nt

TodoItem(title='Profit', id=2, done=True)

# Export -

In [41]:
#|hide
import nbdev; nbdev.nbdev_export()